In [57]:
# crawl through the root directory and count number of files in folder GoPro
import os
import os.path
import re
import subprocess
import json

root_dir = '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/' # 2023 August
# root_dir = '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/North_Garden/May_2024/May24_updated_structure/ego_camera/' # 2024 May
# root_dir = '/standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/North_Garden/Sep_2024/Raw/' # 2024 September

output_csv = './Aug2023_video_stats.csv'


In [66]:
import os
import csv
import subprocess
import json
import re

def get_stats(root_dir, output_csv):
    count = 0
    # Open CSV file for writing
    with open(output_csv, mode='w', newline='') as csv_file:
        fieldnames = ['filename', 'duration', 'total_frames']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        for root, dirs, files in os.walk(root_dir):
            for file in files:
                if re.match(r'.*\.avi$', file, re.IGNORECASE): #avi for smartglass (Aug 2023), MP4 for GoPro (May,Sep 2024)
                    # Get full path of the file
                    file_path = os.path.join(root, file)
                    # duration, num_frames = get_video_info(file_path) # for ffprobe
                    duration, num_frames = get_video_info_opencv(file_path) # for opencv
                    
                    if duration is not None and num_frames is not None:
                        print(f"Processing {file_path}: Duration = {duration} seconds, Frames = {num_frames}")
                        
                        # Write the video info to the CSV file
                        writer.writerow({
                            'filename': file_path,
                            'duration': duration,
                            'total_frames': num_frames
                        })
                        
                    else:
                        print(f"Could not retrieve video information for {file_path}")
                    
                    count += 1
    
    return count

def get_video_info(file_path):
    # Use ffprobe to get video information
    command = [
        'ffprobe',
        '-v', 'error',
        '-select_streams', 'v:0',
        '-count_packets',
        '-show_entries', 'stream=duration,nb_frames',
        '-of', 'json',
        file_path
    ]
    try:
        print(f"Fetching info for {file_path}")
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        video_info = json.loads(result.stdout)
        
        if 'streams' in video_info and len(video_info['streams']) > 0:
            duration = float(video_info['streams'][0].get('duration', 0))
            num_frames = int(video_info['streams'][0].get('nb_frames', 0))
            return duration, num_frames
        else:
            return None, None
    except Exception as e:
        print(f"Error while fetching video info: {e}")
        return None, None


import cv2

def get_video_info_opencv(file_path):
    try:
        # Open the video file
        cap = cv2.VideoCapture(file_path)
        
        if not cap.isOpened():
            print(f"Error: Could not open video file {file_path}")
            return None, None

        # Get total number of frames
        num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Get the frame rate (frames per second)
        fps = cap.get(cv2.CAP_PROP_FPS)

        
        # Calculate duration in seconds
        if fps > 0 and num_frames > 0:
            duration = num_frames / fps
        elif fps > 0 and num_frames == 0:
            # Manually count the frames
            num_frames = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                num_frames += 1
        
        # Calculate duration in seconds
        duration = num_frames / fps
        # Release the video capture object
        cap.release()
        
        return duration, num_frames
    except Exception as e:
        print(f"Error while fetching video info for {file_path}: {e}")
        return None, None



In [67]:
# number of GoPro recordings in the root directory
file_count = get_stats(root_dir, output_csv)
print(f"Processed {file_count} video files. Stats saved to {output_csv}.")


Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/oxygen-bvm/14-08-2023-14-13-12.avi: Duration = 16.6 seconds, Frames = 415
Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/oxygen-bvm/19-10-2023-12-50-31.avi: Duration = 12.7 seconds, Frames = 381
Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/oxygen-bvm/14-08-2023-14-12-02.avi: Duration = 18.0 seconds, Frames = 450


[mjpeg @ 0x177e380] error count: 268435463
[mjpeg @ 0x177e380] error y=29 x=39
[mjpeg @ 0x177e380] error count: 268435463
[mjpeg @ 0x177e380] error y=29 x=39


Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/oxygen-bvm/19-10-2023-12-51-03.avi: Duration = 14.0 seconds, Frames = 420


[mjpeg @ 0x1bded80] overread 8


Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/oxygen-bvm/29-08-2023-20-35-49.avi: Duration = 24.7 seconds, Frames = 741
Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/oxygen-bvm/14-08-2023-14-08-03.avi: Duration = 22.88 seconds, Frames = 572


...

Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/defib_pads_attachment/14-08-2023-14-25-19.avi: Duration = 8.633333333333333 seconds, Frames = 259


[mjpeg @ 0x1c06d00] overread 8


Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/defib_pads_attachment/14-08-2023-14-22-30.avi: Duration = 12.266666666666667 seconds, Frames = 368
Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/defib_pads_attachment/19-10-2023-12-59-41.avi: Duration = 11.533333333333333 seconds, Frames = 346


[mjpeg @ 0x1c06d00] overread 8


Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/defib_pads_attachment/29-08-2023-20-51-38.avi: Duration = 123.73333333333333 seconds, Frames = 3712
Processing /standard/UVA-DSA/NIST EMS Project Data/CognitiveEMS_Datasets/EMS_Interventions/videos/defib_pads_attachment/19-10-2023-12-56-28.avi: Duration = 11.6 seconds, Frames = 348
Processed 74 video files. Stats saved to ./Aug2023_video_stats.csv.


# Separate LL NG

In [68]:
# read csv
import pandas as pd
df = pd.read_csv(output_csv)

In [74]:
# filter files that has -08- in the filename
LL_aug_23_df = df[df['filename'].str.contains('-08-')]
print(f"Found {LL_aug_23_df.shape[0]} files with -08- in the filename")
LL_aug_23_df.to_csv('./LL_aug_23.csv', index=False)

# filter files that has -10- in the filename
NG_sep_23_df = df[df['filename'].str.contains('-10-')]
print(f"Found {NG_sep_23_df.shape[0]} files with -10- in the filename")
NG_sep_23_df.to_csv('./NG_sep_23.csv', index=False)


# filter files that scenario in the filename
LL_scenarios_aug_23_df = df[df['filename'].str.contains('scenario')]
print(f"Found {LL_scenarios_aug_23_df.shape[0]} files with scenario in the filename")
LL_scenarios_aug_23_df.to_csv('./LL_scenarios_aug_23.csv', index=False)

Found 45 files with -08- in the filename
Found 22 files with -10- in the filename
Found 8 files with scenario in the filename
